In [3]:
pip install matplotlib-venn

In [4]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [5]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.3_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.3) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.3) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.8 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31631 sha256=99364df3c4495cc221b970a171cd7182dd495b59108507d2a79a79968eb94204
  Stored in directory: /root/.cache/pip/wheels/32/98/bd/4893d6923dd027f455b250367d402bfd69a6f4416581df46db
Successfully built libarchive


In [6]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [7]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 92.3 MB/s eta 0:00:00


In [9]:
!pip install stable_baselines3

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [10]:
import numpy as np
import pandas as pd
import gym
from gym import spaces
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy


In [11]:
# Step 1: Define a custom Gym environment
class NodeStateEnv(gym.Env):
    def __init__(self, num_nodes=100):
        super(NodeStateEnv, self).__init__()
        self.num_nodes = num_nodes
        self.current_step = 0
        self.max_steps = 100  # Maximum number of steps per episode

        # Define action and observation space
        # Actions: 0 = set node to active, 1 = set node to inactive, 2 = set node to sleep
        self.action_space = spaces.Discrete(3)  # 3 possible actions
        self.observation_space = spaces.Box(low=0, high=100, shape=(num_nodes, 2), dtype=np.float32)  # Node features: traffic_load, energy_level

        # Initialize node states and features
        self.node_states = np.random.choice(["active", "inactive", "sleep"], num_nodes)
        self.node_features = np.random.rand(num_nodes, 2) * 100  # Random traffic_load and energy_level

    def reset(self):
        # Reset the environment to initial state
        self.current_step = 0
        self.node_states = np.random.choice(["active", "inactive", "sleep"], self.num_nodes)
        self.node_features = np.random.rand(self.num_nodes, 2) * 100
        return self.node_features

    def step(self, action):
        # Execute one step in the environment
        self.current_step += 1

        # Apply action to a random node (for simplicity)
        node_id = np.random.randint(0, self.num_nodes)
        self.node_states[node_id] = ["active", "inactive", "sleep"][action]

        # Calculate reward (example: maximize energy efficiency and minimize delay)
        energy_consumption = np.sum(self.node_features[:, 1] * (self.node_states == "active"))
        delay = np.sum(self.node_features[:, 0] * (self.node_states == "inactive"))
        reward = -energy_consumption - delay  # Reward function to minimize energy and delay

        # Check if the episode is done
        done = self.current_step >= self.max_steps

        # Return observation, reward, done, and info
        return self.node_features, reward, done, {}

    def render(self, mode="human"):
        # Optional: Render the environment (not implemented here)
        pass

In [12]:
# Step 2: Create the environment
env = NodeStateEnv(num_nodes=100)


In [14]:
!pip install 'shimmy>=2.0'

In [15]:
# Step 3: Initialize the DQN model
model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.001, buffer_size=10000, batch_size=64, gamma=0.99)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [16]:
# Step 4: Train the DQN model
model.learn(total_timesteps=100000)

-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 100       |
|    ep_rew_mean      | -3.22e+05 |
|    exploration_rate | 0.962     |
| time/               |           |
|    episodes         | 4         |
|    fps              | 571       |
|    time_elapsed     | 0         |
|    total_timesteps  | 400       |
| train/              |           |
|    learning_rate    | 0.001     |
|    loss             | 321       |
|    n_updates        | 74        |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 100       |
|    ep_rew_mean      | -3.36e+05 |
|    exploration_rate | 0.924     |
| time/               |           |
|    episodes         | 8         |
|    fps              | 486       |
|    time_elapsed     | 1         |
|    total_timesteps  | 800       |
| train/              |           |
|    learning_rate    | 0.001     |
|    loss             | 132 

In [17]:
# Step 5: Evaluate the trained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: -390424.8756591797, Std reward: 34923.96514660073


In [18]:
# Step 6: Test the trained model
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    if dones:
        obs = env.reset()


In [19]:
model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.001,
    buffer_size=10000,
    batch_size=64,
    gamma=0.99,
    target_update_interval=1000  # Update target network every 1000 steps
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
pip install sb3-contrib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 3.7 MB/s eta 0:00:00


In [21]:
from sb3_contrib import QRDQN
model = QRDQN("MlpPolicy", env, verbose=1, learning_rate=0.001, buffer_size=10000, batch_size=64, gamma=0.99)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.001,
    buffer_size=10000,
    batch_size=64,
    gamma=0.99,
    exploration_fraction=0.1,  # Fraction of timesteps for exploration
    exploration_initial_eps=1.0,  # Initial epsilon value
    exploration_final_eps=0.01  # Final epsilon value
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [23]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

# Stop training when the mean reward reaches a threshold
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=-200, verbose

SyntaxError: incomplete input (<ipython-input-23-bd40b4716aab>, line 4)